In [1]:
import psycopg2

In [2]:
connect = psycopg2.connect(dbname="fnf", host="prd-dt-redshift.conhugwtudej.ap-northeast-2.redshift.amazonaws.com", port=5439, user="wecode", password="Wecode@2022!#")

In [3]:
cursor = connect.cursor()

In [17]:
brand = 'M'
season = "'22S'"
season_py = "'21S'"
season_py2 = "'20S'"
category = '맨투맨'
sub_category = "'후드','맨투맨'"
adult_kids = '성인'
start_dt = '2021-12-12'
end_dt = '2022-02-06'
end_dt_this_week = '2022-02-06'

In [18]:
query = """
with search as (
    select end_dt,
           srch_cnt as srch_cnt_cy,
           0        as srch_cnt_py
    from (
             select distinct b.cat_nm, b.sub_cat_nm, comp_type, comp_brd_nm, kwd_nm
             from (select distinct brd_cd, cat_nm, sub_cat_nm
                   from prcs.db_prdt a
                   where 1 = 1
                     and brd_cd = '{para_brand}'
                     and cat_nm = '{para_category}'
                     and adult_kids_nm = '{para_adult_kids}'
                     and sub_cat_nm  in({para_sub_category})
                  ) a,
                  prcs.db_srch_kwd_naver_mst b
             where (a.cat_nm = b.sub_cat_nm
                 or a.sub_cat_nm = b.sub_cat_nm)
               and a.brd_cd = b.brd_cd
               and adult_kids = '{para_adult_kids}'
               and comp_type = '자사'
         ) a,
         prcs.db_srch_kwd_naver_w b
    where a.kwd_nm = b.kwd
      and b.end_dt between '{para_start_dt}' and '{para_end_dt_this_week}'
    union all
    select end_dt + 364 as end_dt,
           0            as srch_cnt_cy,
           srch_cnt     as srch_cnt_py
    from (
             select distinct b.cat_nm, b.sub_cat_nm, comp_type, comp_brd_nm, kwd_nm
             from (select distinct brd_cd, cat_nm, sub_cat_nm
                   from prcs.db_prdt a
                   where 1 = 1
                     and brd_cd = '{para_brand}'
                     and cat_nm = '{para_category}'
                     and adult_kids_nm = '{para_adult_kids}'
                     and sub_cat_nm  in({para_sub_category})
                  ) a,
                  prcs.db_srch_kwd_naver_mst b
             where (a.cat_nm = b.sub_cat_nm
                 or a.sub_cat_nm = b.sub_cat_nm)
               and a.brd_cd = b.brd_cd
               and adult_kids = '{para_adult_kids}'
               and comp_type = '자사'
         ) a,
         prcs.db_srch_kwd_naver_w b
    where a.kwd_nm = b.kwd
      and b.end_dt between '{para_start_dt}'-364 and '{para_end_dt}'-364
)
, rds as (
    select end_dt
         , sale_nml_qty_cns + sale_ret_qty_cns as sale_qty_kor_ttl
         , 0                                   as sale_qty_kor_ttl_py
         , 0                                   as sale_qty_kor_ttl_py2
         , stor_qty_kor
         , stock_qty                           as stock_qty_kor
         , 0                                   as order_qty
    from prcs.db_scs_w a,
         prcs.db_prdt b
    where a.brd_cd = b.brd_cd
      and a.prdt_cd = b.prdt_cd
      and a.brd_cd = '{para_brand}'
      and cat_nm = '{para_category}'
      and adult_kids_nm = '{para_adult_kids}'
      and sub_cat_nm  in({para_sub_category})
      and a.sesn in ({para_season})
      and end_dt between '{para_start_dt}' and '{para_end_dt_this_week}'
    union all
    select end_dt + 364                        as end_dt
         , 0                                   as sale_qty_kor_ttl
         , sale_nml_qty_cns + sale_ret_qty_cns as sale_qty_kor_ttl_py
         , 0                                   as sale_qty_kor_ttl_py2
         , 0                                   as stor_qty_kor
         , 0                                      asstock_qty
         , 0                                   as order_qty
    from prcs.db_scs_w a,
         prcs.db_prdt b
    where a.brd_cd = b.brd_cd
      and a.prdt_cd = b.prdt_cd
      and a.brd_cd = '{para_brand}'
      and cat_nm = '{para_category}'
      and adult_kids_nm = '{para_adult_kids}'
      and sub_cat_nm  in({para_sub_category})
      and a.sesn in ({para_season_py})
      and end_dt between '{para_start_dt}'-364 and '{para_end_dt}'-364
    union all
    select end_dt + 364 * 2                    as end_dt
         , 0                                   as sale_qty_kor_ttl
         , 0                                   as sale_qty_kor_ttl_py
         , sale_nml_qty_cns + sale_ret_qty_cns as sale_qty_kor_ttl_py2
         , 0                                   as stor_qty_kor
         , 0                                      asstock_qty
         , 0                                   as order_qty
    from prcs.db_scs_w a,
         prcs.db_prdt b
    where a.brd_cd = b.brd_cd
      and a.prdt_cd = b.prdt_cd
      and a.brd_cd = '{para_brand}'
      and cat_nm = '{para_category}'
      and adult_kids_nm = '{para_adult_kids}'
      and sub_cat_nm  in({para_sub_category})
      and a.sesn in ({para_season_py2})
      and end_dt between '{para_start_dt}'-364*2 and '{para_end_dt}'-364*2
    union all
    select date(date_trunc('week', indc_dt_cnfm)) + 6 as end_dt
         , 0
         , 0
         , 0
         , 0
         , 0
         , b.qty                                      as order_qty
    from prcs.dw_ord a,
         prcs.dw_ord_scs b,
         prcs.db_prdt c
    where a.prdt_cd = b.prdt_cd
      and a.po_no = b.po_no
      and a.brd_cd = c.brd_cd
      and a.prdt_cd = c.prdt_cd
      and a.brd_cd = '{para_brand}'
      and cat_nm = '{para_category}'
      and adult_kids_nm = '{para_adult_kids}'
      and sub_cat_nm  in({para_sub_category})
      and a.sesn in ({para_season})
      and apv_stat = 'C'
      and po_cntry in ('A', 'KR')
      and po_cust_cntry != 'M'
      and indc_dt_cnfm between '{para_start_dt}' and '{para_end_dt}'
)
select end_dt
, sum(sale_qty_kor_ttl) as sale_qty_kor_ttl
, sum(sale_qty_kor_ttl_py) as sale_qty_kor_ttl_py
, sum(sale_qty_kor_ttl_py2) as sale_qty_kor_ttl_py2
, sum(stor_qty_kor) as stor_qty_kor
, sum(stock_qty_kor) as stock_qty_kor
, sum(os) as os
, sum(search_qty_cy) as search_qty_cy
, sum(search_qty_py) as search_qty_py
from (
select to_char(end_dt,'yy.mm.dd') as end_dt
     , sum(sale_qty_kor_ttl)     as sale_qty_kor_ttl
     , sum(sale_qty_kor_ttl_py)  as sale_qty_kor_ttl_py
     , sum(sale_qty_kor_ttl_py2) as sale_qty_kor_ttl_py2
     , sum(stor_qty_kor)         as stor_qty_kor
     , sum(stock_qty_kor)        as stock_qty_kor
     , sum(order_qty)            as os
     , 0 as search_qty_cy
     , 0 as search_qty_py
from rds
group by end_dt
union all
select to_char(end_dt, 'yy.mm.dd') as end_dt
     , 0,0,0,0,0,0
     , sum(srch_cnt_cy)            as search_qty_cy
     , sum(srch_cnt_py)            as search_qty_py
from search
group by end_dt
)
group by end_dt
order by end_dt

        """.format(
            para_brand=brand,
            para_season=season,
            para_season_py=season_py,
            para_season_py2=season_py2,
            para_category=category,
            para_sub_category=sub_category,
            para_adult_kids=adult_kids,
            para_start_dt=start_dt,
            para_end_dt=end_dt,
            para_end_dt_this_week=end_dt_this_week
)

In [19]:
cursor.execute(query)

In [13]:
cursor.execute("ROLLBACK")
connect.commit()

In [20]:
query_output = cursor.fetchall()

In [21]:
query_output

[('21.12.12',
  122,
  498,
  1628,
  23385,
  136167,
  0,
  Decimal('9248.00'),
  Decimal('12139.00')),
 ('21.12.19',
  1227,
  634,
  1321,
  16003,
  150749,
  14908,
  Decimal('8411.00'),
  Decimal('11971.00')),
 ('21.12.26',
  7782,
  1440,
  2454,
  6703,
  149650,
  14800,
  Decimal('7759.00'),
  Decimal('10782.00')),
 ('22.01.02',
  12571,
  3425,
  3060,
  6845,
  142427,
  9200,
  Decimal('7205.00'),
  Decimal('10138.00')),
 ('22.01.09',
  13588,
  11840,
  6594,
  6901,
  131462,
  0,
  Decimal('7314.00'),
  Decimal('9856.00')),
 ('22.01.16',
  9054,
  12888,
  5252,
  9540,
  132542,
  24000,
  Decimal('7426.00'),
  Decimal('9718.00')),
 ('22.01.23',
  9191,
  9534,
  2871,
  16426,
  141205,
  0,
  Decimal('7300.00'),
  Decimal('10317.00')),
 ('22.01.30',
  6342,
  11535,
  3242,
  16960,
  152254,
  9100,
  Decimal('6640.00'),
  Decimal('10521.00')),
 ('22.02.06',
  7257,
  11616,
  2934,
  8380,
  152673,
  0,
  Decimal('8091.00'),
  Decimal('11705.00'))]

In [22]:
from utils.redshift_data import RedshiftData

In [25]:
import pandas.io.sql as psql

In [59]:
redshift_data = RedshiftData(connect, query)

In [60]:
data = redshift_data.get_data()

/opt/homebrew/Caskroom/miniconda/base/envs/fnf/lib/python3.8/site-packages/pandas/io/sql.py:758: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [62]:
data

In [57]:
            result = [{
                "term_cls" : data["term_cls"],                       #해당년도
                "sale_qty_kor_ttl" : data["sale_qty_kor_ttl"],             #판매량 당해
                "sale_qty_kor_ttl_py" : data["sale_qty_kor_ttl_py"],     #판매량 전년
                "sale_qty_kor_ttl_py2" : data["sale_qty_kor_ttl_py2"],                  #판매량 2년전
                "os" : data["os"],       #입고예정(한국)
                "stor_qty_kor" : data["stor_qty_kor"],                     #입고수량(한국)
                "stock_qty_kor" : data["stock_qty_kor"],                       #재고수량(한국)
                "search_qty_cy" : data["search_qty_cy"],                       #검색량 당해
                "search_qty_ㅔy" : data["search_qty_py"],                       #검색량 전년
                }for __, data in data.iterrows()
            ]            

AttributeError: 'list' object has no attribute 'iterrows'

In [36]:
len(data['end_dt'].unique().tolist())

9

In [37]:
start_row = 1
start_col = 1

In [38]:
result=[]

In [39]:
result.append([start_row + 1, start_col, '판매량 당해'])
result.append([start_row + 2, start_col, '판매량 전년'])
result.append([start_row + 3, start_col, '판매량 2년전'])
result.append([start_row + 4, start_col, '입고예정(한국)'])
result.append([start_row + 5, start_col, '입고수량(한국)'])
result.append([start_row + 6, start_col, '재고수량(한국)'])
result.append([start_row + 7, start_col, '검색량 당해'])
result.append([start_row + 8, start_col, '검색량 전년'])

In [40]:
result

[[2, 1, '판매량 당해'],
 [3, 1, '판매량 전년'],
 [4, 1, '판매량 2년전'],
 [5, 1, '입고예정(한국)'],
 [6, 1, '입고수량(한국)'],
 [7, 1, '재고수량(한국)'],
 [8, 1, '검색량 당해'],
 [9, 1, '검색량 전년']]

In [41]:
for i, row in data.iterrows():
    result.append([start_row, start_col + i + 1, row['end_dt']])
    result.append([start_row + 1, start_col + i + 1, row['sale_qty_kor_ttl']])
    result.append([start_row + 2, start_col + i + 1, row['sale_qty_kor_ttl_py']])
    result.append([start_row + 3, start_col + i + 1, row['sale_qty_kor_ttl_py2']])
    result.append([start_row + 4, start_col + i + 1, row['os']])
    result.append([start_row + 5, start_col + i + 1, row['stor_qty_kor']])
    result.append([start_row + 6, start_col + i + 1, row['stock_qty_kor']])
    result.append([start_row + 7, start_col + i + 1, row['search_qty_cy']])
    result.append([start_row + 8, start_col + i + 1, row['search_qty_py']])

In [43]:
result

[[2, 1, '판매량 당해'],
 [3, 1, '판매량 전년'],
 [4, 1, '판매량 2년전'],
 [5, 1, '입고예정(한국)'],
 [6, 1, '입고수량(한국)'],
 [7, 1, '재고수량(한국)'],
 [8, 1, '검색량 당해'],
 [9, 1, '검색량 전년'],
 [1, 2, '21.12.12'],
 [2, 2, 122],
 [3, 2, 498],
 [4, 2, 1628],
 [5, 2, 0],
 [6, 2, 23385],
 [7, 2, 136167],
 [8, 2, 9248.0],
 [9, 2, 12139.0],
 [1, 3, '21.12.19'],
 [2, 3, 1227],
 [3, 3, 634],
 [4, 3, 1321],
 [5, 3, 14908],
 [6, 3, 16003],
 [7, 3, 150749],
 [8, 3, 8411.0],
 [9, 3, 11971.0],
 [1, 4, '21.12.26'],
 [2, 4, 7782],
 [3, 4, 1440],
 [4, 4, 2454],
 [5, 4, 14800],
 [6, 4, 6703],
 [7, 4, 149650],
 [8, 4, 7759.0],
 [9, 4, 10782.0],
 [1, 5, '22.01.02'],
 [2, 5, 12571],
 [3, 5, 3425],
 [4, 5, 3060],
 [5, 5, 9200],
 [6, 5, 6845],
 [7, 5, 142427],
 [8, 5, 7205.0],
 [9, 5, 10138.0],
 [1, 6, '22.01.09'],
 [2, 6, 13588],
 [3, 6, 11840],
 [4, 6, 6594],
 [5, 6, 0],
 [6, 6, 6901],
 [7, 6, 131462],
 [8, 6, 7314.0],
 [9, 6, 9856.0],
 [1, 7, '22.01.16'],
 [2, 7, 9054],
 [3, 7, 12888],
 [4, 7, 5252],
 [5, 7, 24000],
 [6, 7, 9540],
 [7, 7

In [44]:
cols = len(data['end_dt'].unique().tolist())

In [45]:
dic = {}

In [47]:
dic['result'] = result

In [49]:
dic['cols'] = cols

In [50]:
dic

{'result': [[2, 1, '판매량 당해'],
  [3, 1, '판매량 전년'],
  [4, 1, '판매량 2년전'],
  [5, 1, '입고예정(한국)'],
  [6, 1, '입고수량(한국)'],
  [7, 1, '재고수량(한국)'],
  [8, 1, '검색량 당해'],
  [9, 1, '검색량 전년'],
  [1, 2, '21.12.12'],
  [2, 2, 122],
  [3, 2, 498],
  [4, 2, 1628],
  [5, 2, 0],
  [6, 2, 23385],
  [7, 2, 136167],
  [8, 2, 9248.0],
  [9, 2, 12139.0],
  [1, 3, '21.12.19'],
  [2, 3, 1227],
  [3, 3, 634],
  [4, 3, 1321],
  [5, 3, 14908],
  [6, 3, 16003],
  [7, 3, 150749],
  [8, 3, 8411.0],
  [9, 3, 11971.0],
  [1, 4, '21.12.26'],
  [2, 4, 7782],
  [3, 4, 1440],
  [4, 4, 2454],
  [5, 4, 14800],
  [6, 4, 6703],
  [7, 4, 149650],
  [8, 4, 7759.0],
  [9, 4, 10782.0],
  [1, 5, '22.01.02'],
  [2, 5, 12571],
  [3, 5, 3425],
  [4, 5, 3060],
  [5, 5, 9200],
  [6, 5, 6845],
  [7, 5, 142427],
  [8, 5, 7205.0],
  [9, 5, 10138.0],
  [1, 6, '22.01.09'],
  [2, 6, 13588],
  [3, 6, 11840],
  [4, 6, 6594],
  [5, 6, 0],
  [6, 6, 6901],
  [7, 6, 131462],
  [8, 6, 7314.0],
  [9, 6, 9856.0],
  [1, 7, '22.01.16'],
  [2, 7, 9054],
  [

In [55]:
result

[[2, 1, '판매량 당해'],
 [3, 1, '판매량 전년'],
 [4, 1, '판매량 2년전'],
 [5, 1, '입고예정(한국)'],
 [6, 1, '입고수량(한국)'],
 [7, 1, '재고수량(한국)'],
 [8, 1, '검색량 당해'],
 [9, 1, '검색량 전년'],
 [1, 2, '21.12.12'],
 [2, 2, 122],
 [3, 2, 498],
 [4, 2, 1628],
 [5, 2, 0],
 [6, 2, 23385],
 [7, 2, 136167],
 [8, 2, 9248.0],
 [9, 2, 12139.0],
 [1, 3, '21.12.19'],
 [2, 3, 1227],
 [3, 3, 634],
 [4, 3, 1321],
 [5, 3, 14908],
 [6, 3, 16003],
 [7, 3, 150749],
 [8, 3, 8411.0],
 [9, 3, 11971.0],
 [1, 4, '21.12.26'],
 [2, 4, 7782],
 [3, 4, 1440],
 [4, 4, 2454],
 [5, 4, 14800],
 [6, 4, 6703],
 [7, 4, 149650],
 [8, 4, 7759.0],
 [9, 4, 10782.0],
 [1, 5, '22.01.02'],
 [2, 5, 12571],
 [3, 5, 3425],
 [4, 5, 3060],
 [5, 5, 9200],
 [6, 5, 6845],
 [7, 5, 142427],
 [8, 5, 7205.0],
 [9, 5, 10138.0],
 [1, 6, '22.01.09'],
 [2, 6, 13588],
 [3, 6, 11840],
 [4, 6, 6594],
 [5, 6, 0],
 [6, 6, 6901],
 [7, 6, 131462],
 [8, 6, 7314.0],
 [9, 6, 9856.0],
 [1, 7, '22.01.16'],
 [2, 7, 9054],
 [3, 7, 12888],
 [4, 7, 5252],
 [5, 7, 24000],
 [6, 7, 9540],
 [7, 7